# Inference notebook (only for submissions)

In [1]:
!pip install ../input/efficientnet-pytorch
!pip install "../input/pretrained-models/pretrained-models.pytorch-master"


Processing /kaggle/input/efficientnet-pytorch
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=20193 sha256=fcacc5a19b1b0263fb8a0a6ca17dfe61d9a4eee7b59bcdfd39ef2d95fe4d8013
  Stored in directory: /root/.cache/pip/wheels/c4/11/46/c78f049f947bbd6a0dc92e74951904c9b2fa9192864ed474dd
Successfully built efficientnet-pytorch
Processing /kaggle/input/pretrained-models/pretrained-models.pytorch-master
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=62047 sha256=c20bed9e28956da382dc96b209bd2231e8fe2c32edc3bd94dd0f2e9a2012e89b
  Stored in directory: /root/.cache/pip/wheels/ea/01/4e/3421adc08dbbe88fd381b4372f4144c20d3e63088750d11bf2
Successfully built pretrainedmodels


In [2]:
from fastai.vision.all import *
from efficientnet_pytorch import EfficientNet
import albumentations
#import torchvision.models as pretrainedmodels

# Submission setup

In [3]:
class SubmissionConfig:
    n_tta       = 1
    beta        = 0.25
    models      = ['model-f0.pkl', 'model-f1.pkl', 'model-f2.pkl', 'model-f3.pkl', 'model-f4.pkl']
    
cfg = SubmissionConfig()

In [4]:
# this is only for submissions
path_str = '../input/cassava-leaf-disease-classification'

submission_df = pd.read_csv(f'{path_str}/sample_submission.csv')

models_path = '../input/effnetmodels/'
test_images_path = f'{path_str}/test_images/'
test_data_path = submission_df['image_id'].apply(lambda x: test_images_path+x)

# ALL COMBINED[](http://)

In [5]:
all_album = [albumentations.Resize(512,512),albumentations.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225],
            ),albumentations.Transpose(p=1.),albumentations.HorizontalFlip(p=1.),
            albumentations.VerticalFlip(p=1.),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=1.
            ),]

In [6]:
len(all_album)-2

4

In [7]:
final_predictions = 0
for i in range(4):
    class AlbumentationsTransform(RandTransform):
        split_idx,order = None, 2
    
        def __init__(self, train_aug, valid_aug): 
            store_attr()
    
        def before_call(self, b, split_idx):
            self.idx = split_idx
    
        def encodes(self, img: PILImage):
            if self.idx == 0:
                aug_img = self.train_aug(image=np.array(img))['image']
            else:
                aug_img = self.valid_aug(image=np.array(img))['image']
            return PILImage.create(aug_img)
    
    def get_train_aug(size):
        return albumentations.Compose([all_album[0],all_album[1],all_album[i+2]], p=1.)
    
    def get_valid_aug(size): 
        return albumentations.Compose([
            albumentations.Resize(size, size),
            albumentations.CenterCrop(size, size, p=1.),
    ], p=1.)


    def get_x(row): return images_path/row['image_id']
    def get_y(row): return row['label']
    
    
    predictions = 0

    for model in cfg.models:
        learn = load_learner(Path(models_path + model), cpu=False).to_fp16()
        learn_tst_dl = learn.dls.test_dl(test_data_path)

        learn.cbs.pop(3) # remove wandb
        learn.cbs.pop(3) # remove cutmix
        learn.cbs.pop(3) # remove savemodel callback
    
        learn_predictions = learn.tta(dl=learn_tst_dl, n=cfg.n_tta, beta=cfg.beta)
        predictions += learn_predictions[0]

    

    predictions = predictions / len(cfg.models)
    final_predictions += predictions
    
final_predictions = final_predictions/(len(all_album)-2)

    
    

In [8]:
path_str = '../input/cassava-leaf-disease-classification'
models_path = '../input/resnextmodels/'
test_images_path = f'{path_str}/test_images/'


In [9]:
final_predictions_resnext = 0
for i in range(4):
    class AlbumentationsTransform(RandTransform):
        split_idx,order = None, 2
    
        def __init__(self, train_aug, valid_aug): 
            store_attr()
    
        def before_call(self, b, split_idx):
            self.idx = split_idx
    
        def encodes(self, img: PILImage):
            if self.idx == 0:
                aug_img = self.train_aug(image=np.array(img))['image']
            else:
                aug_img = self.valid_aug(image=np.array(img))['image']
            return PILImage.create(aug_img)
    
    def get_train_aug(size): 
        return albumentations.Compose([all_album[0],all_album[1],all_album[i+2]], p=1.)
    
    def get_valid_aug(size): 
        return albumentations.Compose([
            albumentations.Resize(size, size),
            albumentations.CenterCrop(size, size, p=1.),
    ], p=1.)


    def get_x(row): return images_path/row['image_id']
    def get_y(row): return row['label']
    
    
    predictions = 0

    for model in cfg.models:
        learn = load_learner(Path(models_path + model), cpu=False).to_fp16()
        learn_tst_dl = learn.dls.test_dl(test_data_path)

        learn.cbs.pop(3) # remove wandb
        learn.cbs.pop(3) # remove cutmix
        learn.cbs.pop(3) # remove savemodel callback
    
        learn_predictions = learn.tta(dl=learn_tst_dl, n=cfg.n_tta, beta=cfg.beta)
        predictions += learn_predictions[0]

    

    predictions = predictions / len(cfg.models)
    final_predictions_resnext += predictions
    
final_predictions_resnext = final_predictions_resnext/(len(all_album)-2)

    


In [10]:
final_final_predictions = (0.5*final_predictions) + (0.5*final_predictions_resnext)

# Augmentations for inference 

In [11]:
"""
class AlbumentationsTransform(RandTransform):
    split_idx,order = None, 2
    
    def __init__(self, train_aug, valid_aug): 
        store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)
    
def get_train_aug(size): 
    return albumentations.Compose([
            albumentations.RandomResizedCrop(size,size),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
], p=1.)
def get_valid_aug(size): 
    return albumentations.Compose([
        albumentations.Resize(size, size),
        albumentations.CenterCrop(size, size, p=1.),
], p=1.)


def get_x(row): return images_path/row['image_id']
def get_y(row): return row['label']
"""

"\nclass AlbumentationsTransform(RandTransform):\n    split_idx,order = None, 2\n    \n    def __init__(self, train_aug, valid_aug): \n        store_attr()\n    \n    def before_call(self, b, split_idx):\n        self.idx = split_idx\n    \n    def encodes(self, img: PILImage):\n        if self.idx == 0:\n            aug_img = self.train_aug(image=np.array(img))['image']\n        else:\n            aug_img = self.valid_aug(image=np.array(img))['image']\n        return PILImage.create(aug_img)\n    \ndef get_train_aug(size): \n    return albumentations.Compose([\n            albumentations.RandomResizedCrop(size,size),\n            albumentations.Transpose(p=0.5),\n            albumentations.HorizontalFlip(p=0.5),\n            albumentations.VerticalFlip(p=0.5),\n            albumentations.ShiftScaleRotate(p=0.5),\n            albumentations.HueSaturationValue(\n                hue_shift_limit=0.2, \n                sat_shift_limit=0.2, \n                val_shift_limit=0.2, \n         

# Prediction loop

In [12]:
"""
predictions = 0

for model in cfg.models:
    learn = load_learner(Path(models_path + model), cpu=False).to_fp16()
    learn_tst_dl = learn.dls.test_dl(test_data_path)

    learn.cbs.pop(3) # remove wandb
    learn.cbs.pop(3) # remove cutmix
    learn.cbs.pop(3) # remove savemodel callback
    
    learn_predictions = learn.tta(dl=learn_tst_dl, n=cfg.n_tta, beta=cfg.beta)
    predictions += learn_predictions[0]

    
# average the predictions
predictions = predictions / len(cfg.models)
"""

'\npredictions = 0\n\nfor model in cfg.models:\n    learn = load_learner(Path(models_path + model), cpu=False).to_fp16()\n    learn_tst_dl = learn.dls.test_dl(test_data_path)\n\n    learn.cbs.pop(3) # remove wandb\n    learn.cbs.pop(3) # remove cutmix\n    learn.cbs.pop(3) # remove savemodel callback\n    \n    learn_predictions = learn.tta(dl=learn_tst_dl, n=cfg.n_tta, beta=cfg.beta)\n    predictions += learn_predictions[0]\n\n    \n# average the predictions\npredictions = predictions / len(cfg.models)\n'

# Submit

In [13]:
submission_df['label'] = np.argmax(final_final_predictions, axis=1)
submission_df.to_csv('submission.csv', index=False)

In [14]:
submission_df

,image_id,label
0,2216849948.jpg,4
